In [1]:
using Revise 

using Graphs
using Random
using SimpleWeightedGraphs
using SparseArrays
using StatsBase

In [2]:
# USE THESE PACKAGES FOR NOW

# load IterativeHeaps
using Pkg
Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/IterativeHeaps.jl")
using IterativeHeaps

# load GraphDistanceAlgorithms
Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/GraphDistanceAlgorithms.jl")
using GraphDistanceAlgorithms

# get the graph fragments package
Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/GraphFragments.jl/")
using GraphFragments

# get the DiscreteGraphAlgorithms package
Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/DiscreteGraphAlgorithms.jl/")
using DiscreteGraphAlgorithms

   Resolving package versions...
  No Changes to `~/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/Project.toml`
  No Changes to `~/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/Manifest.toml`
Precompiling IterativeHeaps
  ✓ IterativeHeaps
  1 dependency successfully precompiled in 2 seconds. 2 already precompiled.
   Resolving package versions...
  No Changes to `~/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/Project.toml`
  No Changes to `~/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/Manifest.toml`
Precompiling GraphDistanceAlgorithms
        Info Given GraphDistanceAlgorithms was explicitly requested, output will be shown live 
   Resolving package versions...
  No Changes to `~/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/Project.toml`
  No Changes to `~/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/Manifest.toml`
  ✓ GraphDistanceAlgorithms
  1 dependency successfully precompiled in 4 seconds. 23 already precompi

In [3]:
# use this as a test
fp_egl = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists/dprk_iran_edges.egl"
graph_wrapper_dprk_undir = read_egl(fp_egl);
#graph = copy(graph_wrapper_dprk_undir.graph)


j, i, w = findnz(graph_wrapper_dprk_undir.A)
w = Random.rand(length(w))
graph = SimpleWeightedGraph(j, i, w, );





In [15]:
function testme(
    x::Float64;
    this_one::Symbol = :you,
    kwargs...
)
    
    out = (this_one == :you) ? (x + 9) : (x - 12.4)
    
    return out
end


function tm2(
    x::Float64;
    kwargs...
)   
    
    out = testme(x/2; kwargs...)
    
    return out
end
        

tm2 (generic function with 1 method)

In [22]:
@time tm2(4.4; nobody = 4, this_one = :you, eyo_4 = :thisisntit,)


  0.000001 seconds


11.2

In [7]:
@time neighbors(graph, 21)



  0.000017 seconds (1 allocation: 48 bytes)


┌ Error: Failed to revise /Users/jsyme/Documents/Projects/git_jbus/DiscreteGraphAlgorithms.jl/src/Algorithms/GraphOptimizationIterators.jl
│   exception =
│    invalid redefinition of constant DiscreteGraphAlgorithms.OptimizationParameters
│    Stacktrace:
│     [1] top-level scope
│       @ none:0
│    Revise evaluation error at /Users/jsyme/Documents/Projects/git_jbus/CommunityDetectionAlgorithms/none:0
│    
└ @ Revise ~/.julia/packages/Revise/bAgL0/src/packagedef.jl:722
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /Users/jsyme/Documents/Projects/git_jbus/DiscreteGraphAlgorithms.jl/src/Algorithms/GraphOptimizationIterators.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise ~/.julia/packages/Revise/bAgL0/src/packagede

5-element view(::Vector{Int64}, 67:71) with eltype Int64:
 1033
 1859
 1935
 2268
 2275

##  Start building move nodes routine

In [104]:

"""
Create a structure to store some shared vectors that are used 
    repeatedly throughout the calculation of 

"""
struct MoveNodesVectorBundle1
    n::Int64
    vec_comm_membership::Vector{Int64}
    vec_comm_neighbors::Vector{Int64}
    vec_comm_neighbors_added::Vector{Bool}
    vec_comm_size::Vector{Int64}
    vec_comm_weights::Union{Vector{Float64}, Vector{Int64}}
    vec_verts_stable_q::Vector{Bool}
    vec_verts_unstable::Vector{Int64}
    
    function MoveNodesVectorBundle1(
        graph::Union{AbstractGraph, SimpleWeightedGraph};
        vec_comm_membership::VecOrNoth{Int64} = nothing,
        vec_comm_neighbors::VecOrNoth{Int64} = nothing,
        vec_comm_neighbors_added::VecOrNoth{Bool} = nothing,
        vec_comm_size::VecOrNoth{Int64} = nothing,
        vec_comm_weights::Union{VecOrNoth{Float64}, VecOrNoth{Int64}} = nothing,
        vec_verts_stable_q::VecOrNoth{Bool} = nothing,
        vec_verts_unstable::VecOrNoth{Int64} = nothing, # this is the queue
    )
        # initialize sizing parameter
        n = nv(graph)
    
        
        ##  INITIALIZE VECTORS
        
        # community membership; 
        vec_comm_membership = initialize_vector(
            vec_comm_membership, 
            n,
            Int64;
            fill_func = nothing,
        )
        
        vec_comm_neighbors = initialize_vector(vec_comm_neighbors, n, Int64)
        vec_comm_neighbors_added = initialize_vector(vec_comm_neighbors_added, n, Bool)
        vec_comm_size = initialize_vector(vec_comm_size, n, Int64)
        vec_comm_weights = initialize_vector(vec_comm_weights, n, eltype(Graphs.weights(graph)))
        vec_verts_stable_q = initialize_vector(vec_verts_stable_q, n, Bool)
        vec_verts_unstable = initialize_vector(vec_verts_unstable, n, Int64)
        
        
        # update the queue as 1:n 
        vec_verts_unstable .= 1:n
        
        
        # return the output
        return new(
            n,
            vec_comm_neighbors,
            vec_comm_neighbors_added,
            vec_comm_size,
            vec_comm_weights,
            vec_verts_stable_q,
            vec_verts_unstable, 
        )
        
    end
end



"""
Prepare vectors for use in the MoveNodes function

# Constructs

```
prepare_move_nodes_vectors!(
    bundle::MoveNodesVectorBundle1;
)


##  Function Arguments

- `bundle`: MoveNodesVectorBundle1 to update before calculating differences
    and choosing next community
```


"""
function prepare_move_nodes_vectors!(
    bundle::MoveNodesVectorBundle1;
)
    # initialize vector fills
    fill!(bundle.vec_comm_neighbors, 0) # 
    fill!(bundle.vec_comm_neighbors_added, 0) # clear, all are false
    fill!(bundle.vec_comm_size, 0) # all comms set to 0
    fill!(bundle.vec_comm_weights, 0)
    fill!(bundle.vec_verts_stable_q, 0)
    
    # update and shuffle the queue index
    bundle.vec_verts_unstable .= 1:bundle.n
    StatsBase.shuffle!(bundle.vec_verts_unstable)
end



bundle = MoveNodesVectorBundle1(graph)
prepare_move_nodes_vectors!(bundle);




function move_nodes!(
    partition::AbstractPartition,
)



In [113]:
isempty(queue)


false

In [64]:
queue = SimpleQueue2{Int64}(
    119;
    force_unsafe_index_init = true,
    index = StatsBase.shuffle(collect(1:119)),
)

SimpleQueue2{Int64}([63, 64, 34, 51, 11, 33, 37, 98, 87, 67  …  57, 117, 75, 53, 76, 15, 88, 99, 96, 108], [119], 119, 0)

In [122]:
x = collect(1:15)

IterativeHeaps.initialize_vector(x, 15; fill_func = nothing, )


15-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15

In [101]:
@time for i in 1:10
    x[i] = i
end

  0.000006 seconds


In [ ]:
vv = nothing
vv = vv, 10, Bool)



initialize_vector(

10-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [ ]:
function delta_hamiltonian_constant_potts(
    v::Int64,
    vec_membership::Vector{Int64};
    vec_comm_size::Vector
)
    
    adj = neighbors(graph, vertex)
    
    # iterate over neighbors to determine which are in the same comm
    for u in adj
        if vec_membership[u] == vec_membership[v]
            
        end
    end
    
    
end

In [9]:
VecOrNoth{T} = Union{Vector{T}, Nothing}

Union{Nothing, Vector{T}} where T

In [33]:
v .= 1:2

2-element Vector{Real}:
 1
 2

In [6]:
e2 = nothing

for (i, e) in enumerate(edges(graph))
    (i < 10) && println(e)
    (i == 1) && (e2 = e)
end

Edge 3 => 7 with weight 1.1744968244044456
Edge 10 => 11 with weight 1.0207272120483042
Edge 4 => 34 with weight 1.3078873868102796
Edge 45 => 46 with weight 1.4506503952728669
Edge 45 => 47 with weight 0.5627383554134274
Edge 37 => 48 with weight 0.9572967380909618
Edge 14 => 63 with weight 0.710919263528397
Edge 17 => 66 with weight 0.7863439120316822
Edge 22 => 74 with weight 1.3505558013952115


In [ ]:
membs = StatsBase.sample(1:30, nv(graph));

In [316]:
#
@time hamiltonian_constant_potts(
    membs,
    0.0001,
    graph,
)

  0.000645 seconds (15 allocations: 159.828 KiB)


128.34227531160312

In [323]:
"""
"""
struct GraphPartition1
    graph::Union{AbstractGraph, AbstractWeightedGraph}
    element_size::Vector{Int64}
    membership::Vector{Int64}
    
    
    function GraphPartition1(
        graph::Union{AbstractGraph, AbstractWeightedGraph};
        element_size::Union{Vector{Int64}, Nothing} = nothing,
        membership::Union{Vector{Int64}, Nothing} = nothing,
    )
        
        ##  CHECK MEMBERSHIP 
        
        
        if isa(membership, Nothing)
            membership = collect(1:nv(graph))
        end
        
        if size(membership)[1] != nv(graph)
        end
        
        # 
        if 
        
        
        
        return new(
            graph,
            community_size,
            membership,
        )
    end
end

24

In [324]:
v = 2

dict_visited = Dict()
for u in outneighbors(graph, v)
    
    comm = membs[v]
    
    
    
    
end

24
24
24
24
24
24
24


In [328]:
x = collect(1:20)

20-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20

In [128]:
#findall(x -> (x == 2), graph_wrapper_dprk_undir.A.colptr)
#outneighbors(graph_wrapper_dprk_undir.graph, 1)

In [327]:
resize!(x, 30, 9)

LoadError: MethodError: no method matching resize!(::Vector{Int64}, ::Int64, ::Int64)

[0mClosest candidates are:
[0m  resize!(::Vector, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:1312[24m[39m
[0m  resize!([91m::DataFrames.DataFrame[39m, ::Integer)
[0m[90m   @[39m [35mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/dataframe/[39m[90m[4mdataframe.jl:1073[24m[39m
[0m  resize!([91m::MathOptInterface.Nonlinear.ReverseAD.Coloring.IndexedSet[39m, ::Integer)
[0m[90m   @[39m [33mMathOptInterface[39m [90m~/.julia/packages/MathOptInterface/2CULs/src/Nonlinear/ReverseAD/Coloring/[39m[90m[4mColoring.jl:40[24m[39m
[0m  ...


In [329]:
?resize!

search: resize!



```
resize!(a::Vector, n::Integer) -> Vector
```

Resize `a` to contain `n` elements. If `n` is smaller than the current collection length, the first `n` elements will be retained. If `n` is larger, the new elements are not guaranteed to be initialized.

# Examples

```jldoctest
julia> resize!([6, 5, 4, 3, 2, 1], 3)
3-element Vector{Int64}:
 6
 5
 4

julia> a = resize!([6, 5, 4, 3, 2, 1], 8);

julia> length(a)
8

julia> a[1:6]
6-element Vector{Int64}:
 6
 5
 4
 3
 2
 1
```

---

```
resize!(df::DataFrame, n::Integer)
```

Resize `df` to have `n` rows by calling `resize!` on all columns of `df`.

Metadata: this function preserves table-level and column-level `:note`-style metadata.

# Examples

```jldoctest
julia> df = DataFrame(a=1:3, b=4:6)
3×2 DataFrame
 Row │ a      b
     │ Int64  Int64
─────┼──────────────
   1 │     1      4
   2 │     2      5
   3 │     3      6

julia> resize!(df, 2)
2×2 DataFrame
 Row │ a      b
     │ Int64  Int64
─────┼──────────────
   1 │     1      4
   2 │     2      5
```


In [75]:
graph_wrapper_dprk_undir.dims

(2458, 7142)